# DARPA-ARC Notebook 4: fMRI Second Levels

## Precompute Permutations
Based on intial calculations, we assume one full loop of WLS + TFCE will take ~17s. We will submit jobs of 100 iterations (approx. 30 minutes time on cluster).

In [1]:
from my_settings import os, op, np, root_dir, version, n_subj, n_permutations, inc

np.random.seed(47404)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Generate permutations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = []
while True:
    arr = np.random.choice([1,-1],n_subj,replace=True)
    if not np.any(np.apply_along_axis(np.array_equal, 0, permutations, arr)): 
        permutations.append(arr)
    if len(permutations) >= n_permutations: 
        break 
        
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = np.array(permutations)
index = np.arange(0,n_permutations+1,inc)
for n, ix in enumerate(index[1:]):
    np.save(op.join(root_dir, 'fmri_second_levels', 'permutations', '%s_sign_flips_%s' % (version, (n+1))), permutations[ix-inc:ix])
    
with open(op.join(op.join(root_dir, 'fmri_second_levels', '%s_permutations.txt' % version)), 'w') as f:
     f.write('\n'.join(['%i' % i for i in np.arange(n_permutations/inc+1)]))
            
print('Done.')

Done.


## Make Surface Masks

In [2]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, subj_dir, 
                         label_dir, rois, task)
from mne import read_label, read_surface, spatial_tris_connectivity, set_log_level
set_log_level(verbose=False) 

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Make labels.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 

for hemi in ['lh', 'rh']:
    #
    ## Assemble and merge labels.
    label = []
    for roi in rois: label.append(read_label(op.join(label_dir,'%s-%s.label' % (roi, hemi))))
    label = np.sum(label)
    #
    ## Save label.
    label.name = '%s-%s' % (task, hemi)
    label.save(op.join(root_dir, 'fmri_second_levels/%s-%s.label' % (task, hemi)))
    #
    ## Load surface.
    _, tris = read_surface(op.join(subj_dir, 'surf', '%s.white' % hemi))
    mapping = np.in1d(np.unique(tris),label.vertices)
    #
    ## Reduce triangles to those in label.
    ix = np.all(np.apply_along_axis(np.in1d, 0, tris, label.vertices), axis=1)
    tris = tris[ix]
    #
    ## Compute connectivity.
    coo = spatial_tris_connectivity(tris, remap_vertices=True)
    np.savez(op.join(root_dir, 'fmri_second_levels/%s_%s_connectivity' % (version, hemi)), data = coo.data, row = coo.row,
             col = coo.col, shape = coo.shape, mapping=mapping, vertices=label.vertices)
    
print('Done.')

Done.


## Make Volume Mask

In [3]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, subj_dir, 
                         roi_dict, asegf)
                        
from scipy.sparse import coo_matrix

lut = '/usr/local/freesurfer/stable5_3_0/FreeSurferColorLUT.txt'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Create mask.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load aseg.
aseg = nib.load(asegf).get_data()

## Find all voxels in ROI list. Get corresponding labels.
mapping = np.in1d(aseg, list(roi_dict.keys())).reshape(aseg.shape)
voxels = np.where(mapping)
names = np.array([roi_dict[i] for i in aseg[voxels]])
voxels = np.vstack(voxels).T

## Initialize connectivity matrix.
n_voxels, _ = voxels.shape
coo = np.zeros([n_voxels,n_voxels], dtype=int)

## Iteratively test for adjacency.
## Here we use 6-lattice connectivity (up,down,forward,backward,left,right).
for n in range(n_voxels):
    diff = np.linalg.norm(voxels - voxels[n], axis=1)
    M, = np.where(diff==1.)
    for m in M: coo[n,m] = 1 
coo = coo_matrix(coo)
    
## Save.
np.savez(op.join(root_dir, 'fmri_second_levels/%s_mni305_connectivity' % version), data = coo.data, row = coo.row,
         col = coo.col, shape = coo.shape, mapping=mapping, voxels=voxels, names=names)
print('Done.')

Done.


## Extract Mean Signal from ROIs
Necessary for computing percent signal change down the line.

In [4]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir, task,
                         session)

## Define TR onsets.
tr_onsets = np.insert( np.cumsum( np.ones(n_acq - 1) * tr ), 0, 0 )

mean_signal = dict()
for space in ['lh','rh','mni305']:
    #
    print(space)
    #
    ## Load masks.
    npz = np.load(op.join(root_dir,'fmri_second_levels/%s_%s_connectivity.npz' % (version, space)))
    include = npz['mapping']
    #
    ## Preallocate space.
    ms = np.zeros([len(subjects), include.sum()])
    #
    ## Iterate over subjects.
    for n, subject in enumerate(subjects):
        #
        ## Load data.
        subj_dir = op.join(root_dir, 'fmri_first_levels', subject, '%s_%03d' % (task, session), '%03d' % session)
        if space == 'mni305': f = op.join(subj_dir,'fmcpr.sm%s.%s.2mm.b0dc.nii.gz' % (sm, space))
        else: f = op.join(subj_dir,'fmcpr.sm%s.fsaverage.%s.b0dc.nii.gz' % (sm, space))
        data = nib.load(f).get_data()
        #
        ## Censor data. Average across acquisitions.
        try: censor = np.loadtxt(op.join(subj_dir, '%s.censor.%s.par' % (version, fd)))
        except IOError: censor = []
        censor = np.invert(np.in1d(tr_onsets, censor))
        #
        data = data[include,...].squeeze()
        data = data[...,censor].mean(axis=1)
        #
        ## Append.
        ms[n] = data
    #
    ## Store in dictionary.
    mean_signal[space] = ms
    
## Save.
f = op.join(root_dir, 'fmri_second_levels/%s_mean_signal' % version)
np.savez_compressed(f, lh = mean_signal['lh'], rh = mean_signal['rh'], mni305 = mean_signal['mni305'])
print('Done.')

lh
rh
mni305
Done.


## Assemble Data

In [7]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir,
                         concat_sess_dir, thresholds, spaces, models,
                         task, models, conditions_dict)
from mne import read_label

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for model_name, analysis, epochs_type in models:
    #
    for space in spaces:
        #
        ## Load masks.
        npz = np.load(op.join(root_dir, 'fmri_second_levels/%s_%s_connectivity.npz' % (version, space)))
        include = npz['mapping']
        #
        results_dir = op.join(concat_sess_dir, ('%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                (version, task, model_name, 
                                                 analysis, epochs_type, 
                                                 sm, fd, space)))
        #
        for condition in ['Control'] + conditions_dict[analysis]:
            #
            print(model_name, analysis, epochs_type, space, condition)
            #
            condition_dir = op.join(results_dir, ('%s.%s.%s.%s.%s.par' %
                                                  (version, model_name,
                                                   analysis, epochs_type,
                                                   condition)))
            #
            ## Make save directory.
            out_dir = op.join(root_dir, 'fmri_second_levels', ('%s.%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                               (version, task, model_name, 
                                                                analysis, epochs_type,
                                                                sm, fd, space, condition)))
            if not op.isdir(out_dir): os.makedirs(out_dir)
            #
            ## Load data.
            ces = nib.load(op.join(condition_dir, 'ces.nii.gz')).get_data().squeeze()
            cesvar = nib.load(op.join(condition_dir, 'cesvar.nii.gz')).get_data().squeeze()
            affine = nib.load(op.join(condition_dir, 'ces.nii.gz')).affine
            #
            ## Masking.
            ces = ces[include,...]
            cesvar = cesvar[include,...]
            #       
            ## Save.
            np.savez_compressed(op.join(out_dir, 'first_levels'), 
                                ces=ces.squeeze(), cesvar=cesvar.squeeze())
            np.save(op.join(out_dir, 'affine'), affine)
        
print('Done.')

hierarchical PCA VariableEpochs lh Control
hierarchical PCA VariableEpochs lh DDB
hierarchical PCA VariableEpochs lh Risk
hierarchical PCA VariableEpochs lh Reward
hierarchical PCA VariableEpochs rh Control
hierarchical PCA VariableEpochs rh DDB
hierarchical PCA VariableEpochs rh Risk
hierarchical PCA VariableEpochs rh Reward
hierarchical PCA VariableEpochs mni305 Control
hierarchical PCA VariableEpochs mni305 DDB
hierarchical PCA VariableEpochs mni305 Risk
hierarchical PCA VariableEpochs mni305 Reward
Done.


## Perform WLS Permutations

This is done on on a cluster or in parallel using the fmri_second_levels/wls_perm.csh and fmri_second_levels/wls_perm.py scripts

In [12]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir,
                         concat_sess_dir, thresholds, task, n_subj,
                         X, n_subj, n_pred, prepare_image,
                         load_sparse_coo, wls, spaces, models,
                         conditions_dict)
from mne.stats.cluster_level import _find_clusters as find_clusters

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## I/O parameters.
space = spaces[-1]
model_name, analysis, epochs_type = models[0]
#
print(model_name, analysis, epochs_type, space)
#
## Permutation parameters.
permutations = 0

'''
from subprocess import call # DEBUGGING
regressor = '.'.join([version, model_name, analysis, epochs_type, conditions_dict[analysis][-1], 'par'])
args = [space, regressor, permutations]
call(['python fmri_second_levels/wls_perm.py %s %s %s' % (space, regressor, permutations)], env=os.environ, shell=True)
'''

## TFCE parameters.
threshold = dict(start=0.1, step=0.1, h_power=2, e_power=0.5)
tail = 0
max_step = 1

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

out_dir = op.join(root_dir, 'fmri_second_levels', ('%s.%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                   (version, task, model_name, 
                                                    analysis, epochs_type,
                                                    sm, fd, space, condition)))
## Load data.
npz = np.load(os.path.join(out_dir, 'first_levels.npz'))
ces = npz['ces']
cesvar = np.abs( 1. / npz['cesvar'] )

## Define indices.
connectivity = load_sparse_coo(os.path.join(root_dir, 'fmri_second_levels',
                               '%s_%s_connectivity.npz' % (version, space)))
index,  = np.where(~np.isinf(cesvar).sum(axis=1).astype(bool))
include = ~np.isinf(cesvar).sum(axis=1).astype(bool)
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Setup for permutation testing.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## If specified, load precomputed sign flips.
if permutations: 
    sign_flips = np.load(os.path.join(root_dir, 'fmri_second_levels', 
                                      'permutations',
                                      '%s_sign_flips_%s.npy' % (version, permutations)))
else: 
    sign_flips = np.ones((1,n_subj))
n_shuffles = sign_flips.shape[0]

## Preallocate arrays for results.
shape = [n_shuffles] + list(ces.shape[:-1])
Bmap = np.zeros(shape)
Fmap = np.zeros(shape)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Loop it!
for n, sf in enumerate(sign_flips):
    #
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    #
    for m in index:
        #
        ## Update variables.
        W = np.diag(cesvar[m])
        Y = ces[m]
        #
        ## Permute values.
        ## See Winkler et al. (2014), pg. 385
        ## To compute Hat Matrix, see: https://en.wikipedia.org/wiki/Projection_matrix and 
        Z = X[:,1:]
        ZZ = Z.dot( np.linalg.inv( Z.T.dot(W).dot(Z) ) ).dot(Z.T).dot(W)
        Rz = np.identity(n_subj) - ZZ
        Y = np.diag(sf).dot(Rz).dot(Y)
        #
        ## Perform WLS.
        Bmap[n,m], Fmap[n,m] = wls(X,Y,W) 
    #
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform TFCE.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    #
    _, Fmap[n] = find_clusters(Fmap[n], threshold, tail=tail, connectivity=connectivity, 
                               include=include, max_step=max_step, show_info=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save results.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#  
        
if permutations: 
    f = op.join(out_dir, ('%s.%s.%s.%s.%s.%s.%s.%s.%s_perm-%s' % 
                          (version, task, model_name, analysis,
                           epochs_type, sm, fd, space,
                           condition, permutations)))
else:
    f = op.join(out_dir, ('%s.%s.%s.%s.%s.%s.%s.%s.%s_obs' % 
                          (version, task, model_name, analysis,
                           epochs_type, sm, fd, space, condition)))
np.savez_compressed(f, Bmap=Bmap, Fmap=Fmap)
    
print('Done.')

hierarchical DelibMod VariableEpochs mni305


KeyboardInterrupt: 

## Perform FWE Corrections

In [1]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir,
                         concat_sess_dir, thresholds,
                         spaces, task, prepare_image, models,
                         conditions_dict, n_permutations, inc)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = np.arange(int(n_permutations/inc)) + 1

for model_name, analysis, epochs_type in models:
    #
    for condition in ['Control'] + conditions_dict[analysis]:
        #
        for space in spaces:
            #
            print(model_name, analysis, epochs_type, condition, space)
            #
            out_dir = op.join(root_dir, 'fmri_second_levels', ('%s.%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                               (version, task, model_name, 
                                                                analysis, epochs_type,
                                                                sm, fd, space, condition)))
            obs_f = op.join(out_dir, ('%s.%s.%s.%s.%s.%s.%s.%s.%s_obs.npz' % 
                                      (version, task, model_name, analysis,
                                       epochs_type, sm, fd, space, condition)))
            #
            ## Load true effects.
            npz = np.load(obs_f)
            Bmap = npz['Bmap'].squeeze()
            Fmap = npz['Fmap'].squeeze()
            #
            ## Load permutations.
            perm_f = op.join(out_dir, ('%s.%s.%s.%s.%s.%s.%s.%s.%s' %
                                       (version, task, model_name, analysis,
                                        epochs_type, sm, fd, space, condition)) + 
                                       '_perm-%s.npz')
            Pmap = []
            for p in permutations:
                try:
                    npz = np.load(perm_f % p)
                    Pmap.append(npz['Fmap'])
                except Exception as e:
                    print(e, p)  # I was getting bad magic numbers on some file and had to rerun WLS
            Pmap = np.concatenate(Pmap, axis=0)
            n_permutations, _ = Pmap.shape
            #
            ## Compute p-values via FWE.
            p_values = np.ones_like(Fmap)
            for mp in Pmap.max(axis=1): p_values += mp > Fmap
            p_values /= n_permutations + 1.
            p_values = -np.log10(p_values) * np.sign(Bmap)
            #
            ## Save maps.
            out_f = op.join(out_dir, ('%s.%s.%s.%s.%s.%s.%s.%s.%s_fwe' % 
                                      (version, task, model_name, analysis,
                                       epochs_type, sm, fd, space, condition)))
            np.save(out_f, p_values)
            for arr, name in zip([Bmap,Fmap,p_values],['beta', 'F', 'fwe']):
                image = prepare_image(arr, space)
                image = nib.Nifti1Image(image, np.load(op.join(out_dir, 'affine.npy')))
                nib.save(image, op.join(out_dir, '%s.nii.gz' % name))
            
print('Done.')

hierarchical PCA VariableEpochs Control lh
hierarchical PCA VariableEpochs Control rh
hierarchical PCA VariableEpochs Control mni305
hierarchical PCA VariableEpochs DDB lh
hierarchical PCA VariableEpochs DDB rh
hierarchical PCA VariableEpochs DDB mni305
hierarchical PCA VariableEpochs Risk lh
hierarchical PCA VariableEpochs Risk rh
hierarchical PCA VariableEpochs Risk mni305
hierarchical PCA VariableEpochs Reward lh
hierarchical PCA VariableEpochs Reward rh
hierarchical PCA VariableEpochs Reward mni305
Done.


## Perform FDR Corrections
Not used, but programmed for example's sake.

In [4]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir,
                         concat_sess_dir, thresholds, models,
                         spaces, task, prepare_image, conditions_dict)
from mne.stats import fdr_correction

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

permutations = np.arange(int(n_permutations/inc)) + 1

for model_name, analysis, epochs_type in models:
    #
    for condition in ['Control'] + conditions_dict[analysis]:
        #
        FDR, signs = [], []
        #
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute p-values within spaces.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        #
        for n, space in enumerate(spaces):
             #
            print(model_name, analysis, epochs_type, condition, space)
            #
            out_dir = op.join(root_dir, 'fmri_second_levels', ('%s.%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                               (version, task, model_name, 
                                                                analysis, epochs_type,
                                                                sm, fd, space, condition)))
            obs_f = op.join(out_dir, ('%s.%s.%s.%s.%s.%s.%s.%s.%s_obs.npz' % 
                                      (version, task, model_name, analysis,
                                       epochs_type, sm, fd, space, condition)))
            #
            ## Load true effects.
            npz = np.load(obs_f)
            Bmap = npz['Bmap'].squeeze()
            Fmap = npz['Fmap'].squeeze()
            #
            ## Load permutations.
            perm_f = op.join(out_dir, ('%s.%s.%s.%s.%s.%s.%s.%s.%s' %
                                       (version, task, model_name, analysis,
                                        epochs_type, sm, fd, space, condition)) + 
                                       '_perm-%s.npz')
            Pmap = []
            for p in permutations:
                try:
                    npz = np.load(perm_f % p)
                    Pmap.append(npz['Fmap'])
                except Exception as e:
                    print(e, p)  # I was getting bad magic numbers on some file and had to rerun WLS
            Pmap = np.concatenate(Pmap, axis=0)
            n_permutations, _ = Pmap.shape
            #
            ## Compute p-values via FWE.
            p_values = (Pmap >= Fmap).sum(axis=0) + 1.
            p_values /= n_permutations + 1.
            FDR.append(p_values)
            signs.append(np.sign(Bmap))
            #
            ## Save maps.
            for arr, name in zip([Bmap,Fmap],['beta','F']):
                image = prepare_image(arr, space)
                image = nib.Nifti1Image(image, np.load(op.join(out_dir, 'affine.npy')))
                nib.save(image, op.join(out_dir, '%s.nii.gz' % name))        
        #
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Perform FDR corrections.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        #
        ## Assemble info.
        indices = np.concatenate([np.ones_like(arr) * n for n, arr in enumerate(FDR)])
        FDR = np.concatenate(FDR)
        signs = np.concatenate(signs)
        #
        ## Perform FDR correction.
        FDR[np.where(signs)] = fdr_correction(FDR[np.where(signs)])[-1]
        FDR = -np.log10(FDR) * signs
        #
        ## Save maps.
        for n, space in enumerate(spaces):
            out_f = op.join(out_dir, ('%s.%s.%s.%s.%s.%s.%s.%s.%s_fdr' % 
                                      (version, task, model_name, analysis,
                                       epochs_type, sm, fd, space, condition)))
            np.save(out_f, FDR[indices==n])
            image = prepare_image(FDR[indices==n], space)
            image = nib.Nifti1Image(image, np.load(op.join(out_dir,'affine.npy')))
            nib.save(image, op.join(out_dir, 'fdr.nii.gz'))
    
print('Done.')

hierarchical DelibMod VariableEpochs Control lh
hierarchical DelibMod VariableEpochs Control rh
hierarchical DelibMod VariableEpochs Control mni305
hierarchical DelibMod VariableEpochs DDB lh
hierarchical DelibMod VariableEpochs DDB rh
hierarchical DelibMod VariableEpochs DDB mni305
non-hierarchical DelibMod VariableEpochs Control lh
non-hierarchical DelibMod VariableEpochs Control rh
non-hierarchical DelibMod VariableEpochs Control mni305
non-hierarchical DelibMod VariableEpochs DDB lh
non-hierarchical DelibMod VariableEpochs DDB rh
non-hierarchical DelibMod VariableEpochs DDB mni305
hierarchical DelibMod FixedEpochs Control lh
hierarchical DelibMod FixedEpochs Control rh
hierarchical DelibMod FixedEpochs Control mni305
hierarchical DelibMod FixedEpochs DDB lh
hierarchical DelibMod FixedEpochs DDB rh
hierarchical DelibMod FixedEpochs DDB mni305
non-hierarchical DelibMod FixedEpochs Control lh
non-hierarchical DelibMod FixedEpochs Control rh
non-hierarchical DelibMod FixedEpochs Contro

# Section 5: Visualization

## Threshold Second-Level Maps
Thresholding clusters such that:
* p < 0.05 (FWE corrected, alpha = 0.05)
* Surface: clusters > 100mm2
* Volume: clusters > 20 contiguous voxels

In [5]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir,
                         concat_sess_dir, thresholds, models,
                         spaces, task, prepare_image, fs_dir,
                         load_sparse_coo, conditions_dict, psc_threshold)
from mne.stats.cluster_level import _find_clusters as find_clusters

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Thresholding parameters.
threshold = -np.log10( psc_threshold )
min_cluster = dict(lh = 100, rh = 100, mni305 = 20)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for space in spaces:
    #
    ## Load connectivity information.
    connectivity = load_sparse_coo(op.join(root_dir, 'fmri_second_levels/%s_%s_connectivity.npz' % (version, space)))
    #
    ## Load mapping information.
    npz = np.load(op.join(root_dir, 'fmri_second_levels/%s_%s_connectivity.npz' % (version, space)))
    #
    if space != 'mni305':
        vertices = npz['vertices']
        average_area = nib.load(op.join(fs_dir, 'fsaverage', 'surf', '%s.white.avg.area.mgh' % space)).get_data()
        average_area = average_area[vertices].squeeze()
    #
    for model_name, analysis, epochs_type in models:
        #
        print(space, model_name, analysis, epochs_type)
        #
        for condition in ['Control'] + conditions_dict[analysis]:
            #
            ## Load FWE-corrected p-values.
            out_dir = op.join(root_dir, 'fmri_second_levels', ('%s.%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                               (version, task, model_name, 
                                                                analysis, epochs_type,
                                                                sm, fd, space, condition)))
            fwe_f = op.join(out_dir, ('%s.%s.%s.%s.%s.%s.%s.%s.%s_fwe.npy' % 
                                      (version, task, model_name, analysis,
                                       epochs_type, sm, fd, space, condition)))
            fwe = np.load(fwe_f)
            #
            ## Find clusters.
            include = np.where(fwe, True, False)
            clusters, sums = find_clusters(fwe, threshold, tail=0, connectivity=connectivity, 
                                           include=include, t_power=0)
            #
            ## Compute areas.
            if space == 'mni305': 
                cluster_sums = sums
            else:
                cluster_sums = np.array([average_area[c].sum() for c in clusters])
            #
            ## Threshold.
            try:
                survival_ix = np.concatenate([c for c, s in zip(clusters, cluster_sums) if s > min_cluster[space]])
                fwe[~np.in1d(np.arange(fwe.shape[0]), survival_ix)] = 0
            except ValueError:
                #print('No clusters', space, model_name, analysis, epochs_type, condition)
                fwe = np.zeros_like(fwe)
                fwe[0] = 1; fwe[-1] = 1 # pysurfer bug: https://github.com/nipy/PySurfer/issues/267
            #
            ## Save.
            image = prepare_image(fwe, space)
            image = nib.Nifti1Image(image, np.load(op.join(op.dirname(fwe_f),'affine.npy')))
            nib.save(image, op.join(op.dirname(fwe_f), 'fwe_thresh_%s.nii.gz' % psc_threshold))

print('Done.')

lh
lh hierarchical DelibMod VariableEpochs
lh non-hierarchical DelibMod VariableEpochs
lh hierarchical DelibMod FixedEpochs
lh non-hierarchical DelibMod FixedEpochs
lh parameter Risk VariableEpochs
lh parameter Reward VariableEpochs
lh parameter Risk FixedEpochs
lh parameter Reward FixedEpochs
lh hierarchical All VariableEpochs
lh hierarchical All FixedEpochs
lh non-hierarchical All VariableEpochs
lh non-hierarchical All FixedEpochs
lh hierarchical PCA VariableEpochs
rh
rh hierarchical DelibMod VariableEpochs
rh non-hierarchical DelibMod VariableEpochs
rh hierarchical DelibMod FixedEpochs
rh non-hierarchical DelibMod FixedEpochs
rh parameter Risk VariableEpochs
rh parameter Reward VariableEpochs
rh parameter Risk FixedEpochs
rh parameter Reward FixedEpochs
rh hierarchical All VariableEpochs
rh hierarchical All FixedEpochs
rh non-hierarchical All VariableEpochs
rh non-hierarchical All FixedEpochs
rh hierarchical PCA VariableEpochs
mni305
mni305 hierarchical DelibMod VariableEpochs
mni30

## Compute Percent Signal Change

In [6]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir,
                         concat_sess_dir, thresholds, models,
                         spaces, task, session, prepare_image, fs_dir,
                         load_sparse_coo, psc_threshold, subjects,
                         conditions_dict, plt)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Load average signal.
mean_signal = np.load(op.join(root_dir, 'fmri_second_levels/%s_mean_signal.npz' % version))

for space in spaces:
    #
    ## Assemble design matrices.
    subj_dir = op.join(root_dir, 'fmri_first_levels', '%s', '%s_%03d' % (task, session))
    for model_name, analysis, epochs_type in models:
        #
        print(space, model_name, analysis, epochs_type)
        #
        X_f = op.join(subj_dir, 
                      ('%s.%s.%s.%s.%s.%s.%s.%s' % 
                       (version, task, model_name, 
                        analysis, epochs_type,
                        sm, fd, space)), 'X.dat')
        scale_factors = np.array([np.loadtxt(X_f % subject).max(axis=0)[:(len(conditions_dict[analysis])+1)]
                                 for subject in subjects]).T
        #
        for n, condition in enumerate(['Control'] + conditions_dict[analysis]):
            #
            ## Load first levels.
            out_dir = op.join(root_dir, 'fmri_second_levels', ('%s.%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                               (version, task, model_name,
                                                                analysis, epochs_type,
                                                                sm, fd, space, condition)))
            ces = np.load(op.join(out_dir, 'first_levels.npz'))['ces']
            #
            ## Compute PSC (Pernet 2014, Frontiers in Neuroscience).
            ms = np.where(mean_signal[space], mean_signal[space], np.inf).T
            psc = np.divide(ces * scale_factors[n] * 100., ms)
            psc = prepare_image(psc.mean(axis=1), space)
            #
            ## Mask image.
            fwe = nib.load(op.join(out_dir, 'fwe_thresh_%s.nii.gz' % psc_threshold)).get_data()
            psc *= np.where(fwe, 1, 0)
            #
            ## Save.
            image = nib.Nifti1Image(psc, np.load(op.join(out_dir, 'affine.npy')))
            nib.save(image, op.join(out_dir, 'psc.nii.gz'))
        
print('Done.')

lh hierarchical DelibMod VariableEpochs
lh non-hierarchical DelibMod VariableEpochs
lh hierarchical DelibMod FixedEpochs
lh non-hierarchical DelibMod FixedEpochs
lh parameter Risk VariableEpochs
lh parameter Reward VariableEpochs
lh parameter Risk FixedEpochs
lh parameter Reward FixedEpochs
lh hierarchical All VariableEpochs
lh hierarchical All FixedEpochs
lh non-hierarchical All VariableEpochs
lh non-hierarchical All FixedEpochs
lh hierarchical PCA VariableEpochs
rh hierarchical DelibMod VariableEpochs
rh non-hierarchical DelibMod VariableEpochs
rh hierarchical DelibMod FixedEpochs
rh non-hierarchical DelibMod FixedEpochs
rh parameter Risk VariableEpochs
rh parameter Reward VariableEpochs
rh parameter Risk FixedEpochs
rh parameter Reward FixedEpochs
rh hierarchical All VariableEpochs
rh hierarchical All FixedEpochs
rh non-hierarchical All VariableEpochs
rh non-hierarchical All FixedEpochs
rh hierarchical PCA VariableEpochs
mni305 hierarchical DelibMod VariableEpochs
mni305 non-hierarc

## Surface Plots

In [7]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir,
                         concat_sess_dir, thresholds, models,
                         spaces, task, prepare_image, fs_dir,
                         load_sparse_coo, subjects, img_dir,
                         overlay, surface, conditions_dict,
                         psc_threshold)
from surfer import Brain

threshold = -np.log10( psc_threshold )

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for hemi in ['lh', 'rh']: 
    #
    for model_name, analysis, epochs_type in models:
        #
        for condition in ['Control'] + conditions_dict[analysis]:
            #
            print(hemi, model_name, analysis, epochs_type, condition)
            #
            out_dir = op.join(root_dir, 'fmri_second_levels', ('%s.%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                               (version, task, model_name,
                                                                analysis, epochs_type,
                                                                sm, fd, hemi, condition)))
            fn = op.join(out_dir, '%s.nii.gz' % overlay)
            #
            for view in ['lateral', 'medial']:
                #
                brain = Brain('fsaverage', hemi, surface, subjects_dir=fs_dir)
                try: 
                    brain.add_overlay(fn, min=0.04, max=2.5, sign="pos")
                except Exception as e: 
                    print(e)
                    continue
                brain.show_view(view=view)
                od = op.join(img_dir, overlay, surface, 
                             ('%s.%s.%s.%s.%s.%s.%s.' % 
                             (version, task, model_name, 
                              analysis, epochs_type, sm, fd)))
                if not op.isdir(od): 
                    os.makedirs(od)
                out_f = op.join(od, '%s.%s.%s.png' % (condition, hemi, view))
                Brain.save_image(brain, out_f)

lh hierarchical DelibMod VariableEpochs Control
lh hierarchical DelibMod VariableEpochs DDB
lh non-hierarchical DelibMod VariableEpochs Control
lh non-hierarchical DelibMod VariableEpochs DDB
lh hierarchical DelibMod FixedEpochs Control
lh hierarchical DelibMod FixedEpochs DDB
lh non-hierarchical DelibMod FixedEpochs Control
lh non-hierarchical DelibMod FixedEpochs DDB
lh parameter Risk VariableEpochs Control
lh parameter Risk VariableEpochs Risk
lh parameter Reward VariableEpochs Control
lh parameter Reward VariableEpochs Reward
lh parameter Risk FixedEpochs Control
lh parameter Risk FixedEpochs Risk
lh parameter Reward FixedEpochs Control
lh parameter Reward FixedEpochs Reward
lh hierarchical All VariableEpochs Control
lh hierarchical All VariableEpochs DDB
lh hierarchical All VariableEpochs Risk
lh hierarchical All VariableEpochs Reward
lh hierarchical All FixedEpochs Control
lh hierarchical All FixedEpochs DDB
lh hierarchical All FixedEpochs Risk
lh hierarchical All FixedEpochs Rew

## Compute surface summary table

In [10]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir,
                         concat_sess_dir, thresholds, models,
                         spaces, task, prepare_image, fs_dir,
                         load_sparse_coo, subjects, img_dir,
                         overlay, surface, psc_threshold, conditions_dict,
                         label_dir, rois)
from mne import Label, read_label, grow_labels, vertex_to_mni, set_log_level
set_log_level(verbose=False)


threshold = -np.log10( psc_threshold )

## ROI parameters.
extent = 10 #mm
grow = False

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for model_name, analysis, epochs_type in models:
    #
    for condition in ['Control'] + conditions_dict[analysis]:
        #
        for hemi in ['lh', 'rh']:
            #
            print(model_name, analysis, epochs_type, condition, hemi)
            #
            out_dir = op.join(root_dir, 'fmri_second_levels', ('%s.%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                               (version, task, model_name,
                                                                analysis, epochs_type,
                                                                sm, fd, hemi, condition)))
            #label_dir = op.join(root_dir, 'fmri_second_levels/labels/', 'seeds_%s' % task)
            #labels = sorted([f for f in os.listdir(label_dir) if not f.startswith('fig') and f.endswith('label')])
            #
            fmni = open(op.join(label_dir, ('surface_mni2.%s.%s.%s.%s.%s.%s.%s.%s.%s.csv' % 
                                            (version, task, model_name,
                                             analysis, epochs_type,
                                             sm, fd, hemi, condition))), 'w')
            fmni.write(','.join(['Label', 'V', 'X', 'Y', 'Z', 'PSC', 'F', 'p']) + '\n')
            #
            for roi in rois:
                #
                label = read_label(op.join(label_dir, '%s-%s.label' % (roi, hemi)))
                #
                ## Load accompanying overlay.
                f = op.join(out_dir, 'psc.nii.gz')
                overlay = nib.load(f).get_data().squeeze()
                #
                ## Find maximum vertex.
                ix = np.argmax(overlay[label.vertices])
                v = label.vertices[ix]
                #
                ## Extract MNI coordinates.
                x,y,z = vertex_to_mni(v, 0 if hemi == 'lh' else 1, 'fsaverage', fs_dir)[0]
                #
                ## Extract PSC, F-scores, p-values.
                f = op.join(out_dir, 'psc.nii.gz')
                psc = nib.load(f).get_data().squeeze()[v]
                #
                f = op.join(out_dir, 'F.nii.gz')
                F = nib.load(f).get_data().squeeze()[v]
                #
                f = op.join(out_dir, 'fwe_thresh_%.3f.nii.gz' % threshold)
                p = nib.load(f).get_data().squeeze()[v]
                #
                ## Write information.
                fmni.write('%s-%s,%s,%0.0f,%0.0f,%0.0f,%0.2f,%0.2f,%0.6f\n' % (roi, hemi, v, x, y, z, psc, F, 10.**-p))
                #
                if grow:
                    ## Grow label.
                    label = grow_labels('fsaverage', v, extent, 0 if hemi=='lh' else 1, subjects_dir=fs_dir,
                                        names='fig_%s-%s' % (roi, hemi), surface='pial')[0]
                    #
                    ## Ensure label is within actiation. Save.
                    ix = np.in1d(label.vertices, np.where(overlay)[0])
                    label.pos = label.pos[ix]
                    label.values = label.values[ix]
                    label.vertices = label.vertices[ix]
                    label.save('%s/%s.label' % (out_label_dir, label.name))

fmni.close()
print('Done.')

hierarchical DelibMod VariableEpochs Control lh
hierarchical DelibMod VariableEpochs Control rh
hierarchical DelibMod VariableEpochs DDB lh
hierarchical DelibMod VariableEpochs DDB rh
non-hierarchical DelibMod VariableEpochs Control lh
non-hierarchical DelibMod VariableEpochs Control rh
non-hierarchical DelibMod VariableEpochs DDB lh
non-hierarchical DelibMod VariableEpochs DDB rh
hierarchical DelibMod FixedEpochs Control lh
hierarchical DelibMod FixedEpochs Control rh
hierarchical DelibMod FixedEpochs DDB lh
hierarchical DelibMod FixedEpochs DDB rh
non-hierarchical DelibMod FixedEpochs Control lh
non-hierarchical DelibMod FixedEpochs Control rh
non-hierarchical DelibMod FixedEpochs DDB lh
non-hierarchical DelibMod FixedEpochs DDB rh
parameter Risk VariableEpochs Control lh
parameter Risk VariableEpochs Control rh
parameter Risk VariableEpochs Risk lh
parameter Risk VariableEpochs Risk rh
parameter Reward VariableEpochs Control lh
parameter Reward VariableEpochs Control rh
parameter Re

## Compute volume summary table

In [12]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir,
                         concat_sess_dir, thresholds, models,
                         spaces, task, prepare_image, fs_dir,
                         load_sparse_coo, subjects, img_dir, 
                         conditions_dict, psc_threshold, 
                         label_dir)
from nibabel.affines import apply_affine

space = 'mni305'
threshold = -np.log10( psc_threshold )

## ROI parameters.
extent = 6 #mm

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for model_name, analysis, epochs_type in models:
    #
    for condition in ['Control'] + conditions_dict[analysis]:
        #
        print(model_name, analysis, epochs_type, condition)
        #
        out_dir = op.join(root_dir, 'fmri_second_levels', ('%s.%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                           (version, task, model_name,
                                                            analysis, epochs_type,
                                                            sm, fd, space, condition)))
        ## Initialize statistics file.
        fmni = open(op.join(label_dir, ('volume_mni2.%s.%s.%s.%s.%s.%s.%s.%s.%s.csv' % 
                                        (version, task, model_name,
                                         analysis, epochs_type,
                                         sm, fd, hemi, condition))), 'w')
        fmni.write(','.join(['Label','cX','cY','cZ','X','Y','Z','PSC','F','p']) + '\n')
        #
        ## Load data. 
        npz = np.load(op.join(root_dir, 'fmri_second_levels/%s_mni305_connectivity.npz' % version))
        affine = np.load(op.join(out_dir, 'affine.npy'))
        obj = nib.load(op.join(out_dir, 'psc.nii.gz'))
        #
        overlay = obj.get_data().squeeze()
        Fval = nib.load(op.join(out_dir, 'F.nii.gz')).get_data().squeeze()
        pval = nib.load(op.join(out_dir, 'fwe_thresh_%.3f.nii.gz' % threshold)).get_data().squeeze()
        #
        rois = ['Left-Caudate', 'Left-Putamen', 'Left-Hippocampus',
                'Right-Caudate', 'Right-Putamen', 'Right-Hippocampus']
        for roi in rois:
            #
            ## Extract activated voxels in ROI.
            voxels = npz['voxels'][npz['names'] == roi]
            voxels = voxels[np.where(overlay[tuple([arr for arr in voxels.T])])]
            if voxels.shape[0] == 0:
                continue
            #
            ## Find maximally activated voxel.
            ix = np.argmax(overlay[tuple([arr for arr in voxels.T])])
            center = voxels[ix]
            i,j,k = center
            #
            ## Get MNI coordinates.
            x,y,z = apply_affine(affine, center)
            #
            ## Extract max values.
            psc = overlay[i,j,k]
            F = Fval[i,j,k]
            p = pval[i,j,k]
            #
            ## Write to file.
            fmni.write('%s,%0.0d,%0.0d,%0.0d,%0.0d,%0.2d,%0.2d,%0.2f,%0.2f,%0.6f\n' % (roi, i, j, k, x, y, z, psc, F, 10.**-p))
            #
            ## Create sphere: find all voxels within extent.
            dist = [np.linalg.norm( np.diff( apply_affine(affine,np.vstack([center,v])), axis=0 ) ) for v in voxels]
            ix = np.where(np.array(dist)<=extent)
            sphere = voxels[ix]
            #
            ## Save.
            #hemi, roi = roi.split('-')
            #if hemi.startswith('L'): name = '%s-lh' %roi.lower()
            #else: name = '%s-rh' %roi.lower()
            #np.save(op.join(out_dir, name), sphere)
        fmni.close()
print('Done.')

hierarchical DelibMod VariableEpochs Control
hierarchical DelibMod VariableEpochs DDB
non-hierarchical DelibMod VariableEpochs Control
non-hierarchical DelibMod VariableEpochs DDB
hierarchical DelibMod FixedEpochs Control
hierarchical DelibMod FixedEpochs DDB
non-hierarchical DelibMod FixedEpochs Control
non-hierarchical DelibMod FixedEpochs DDB
parameter Risk VariableEpochs Control
parameter Risk VariableEpochs Risk
parameter Reward VariableEpochs Control
parameter Reward VariableEpochs Reward
parameter Risk FixedEpochs Control
parameter Risk FixedEpochs Risk
parameter Reward FixedEpochs Control
parameter Reward FixedEpochs Reward
hierarchical All VariableEpochs Control
hierarchical All VariableEpochs DDB
hierarchical All VariableEpochs Risk
hierarchical All VariableEpochs Reward
hierarchical All FixedEpochs Control
hierarchical All FixedEpochs DDB
hierarchical All FixedEpochs Risk
hierarchical All FixedEpochs Reward
non-hierarchical All VariableEpochs Control
non-hierarchical All Var

## Post-hoc F-statistic Fix
Sam realized very late in the game he should have been saving out the pre-TFCE F-statistics. Fortunately these can be recomputed using the WLS code sans TFCE.

In [15]:
from my_settings import (os, op, np, root_dir, version, n_subj, nib, 
                         sm, fd, tr, n_acq, subjects, subj_dir, read_csv,
                         concat_sess_dir, thresholds, models,
                         spaces, task, prepare_image, fs_dir,
                         load_sparse_coo, subjects, img_dir, wls,
                         X, n_pred, conditions_dict, label_dir)

for model_name, analysis, epochs_type in models:
    #
    for condition in ['Control'] + conditions_dict[analysis]:
        #
        for space in spaces:
            #
            print(model_name, analysis, epochs_type, condition, space)
            #
            out_dir = op.join(root_dir, 'fmri_second_levels', ('%s.%s.%s.%s.%s.%s.%s.%s.%s' % 
                                                               (version, task, model_name,
                                                                analysis, epochs_type,
                                                                sm, fd, space, condition)))
            #
            ## Load data.
            npz = np.load(op.join(out_dir, 'first_levels.npz'))
            ces = npz['ces']
            cesvar = np.abs( 1. / npz['cesvar'] )
            #
            ## Define indices.
            connectivity = load_sparse_coo(op.join(root_dir, 'fmri_second_levels', '%s_%s_connectivity.npz' % (version, space)))
            index,  = np.where(~np.isinf(cesvar).sum(axis=1).astype(bool))
            include = ~np.isinf(cesvar).sum(axis=1).astype(bool)
            #
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Setup for permutation testing.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            #
            sign_flips = np.ones((1,n_subj))
            n_shuffles = sign_flips.shape[0]
            #
            ## Preallocate arrays for results.
            shape = [n_shuffles] + list(ces.shape[:-1])
            Bmap = np.zeros(shape)
            Fmap = np.zeros(shape)
            #
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Main loop.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            #
            ## Loop it!
            for n, sf in enumerate(sign_flips):
                #
                #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
                ### Compute statistics.
                #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
                #
                for m in index:
                    #
                    ## Update variables.
                    W = np.diag(cesvar[m])
                    Y = ces[m]
                    #
                    ## Permute values.
                    ## See Winkler et al. (2014), pg. 385
                    ## To compute Hat Matrix, see: https://en.wikipedia.org/wiki/Projection_matrix and 
                    Z = X[:,1:]
                    ZZ = Z.dot( np.linalg.inv( Z.T.dot(W).dot(Z) ) ).dot(Z.T).dot(W)
                    Rz = np.identity(n_subj) - ZZ
                    Y = np.diag(sf).dot(Rz).dot(Y)
                    #
                    ## Perform WLS.
                    Bmap[n,m], Fmap[n,m] = wls(X,Y,W) 
            #
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            ### Save results.
            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#  
            #
            ## Translate array back into proper space.
            image = prepare_image(Fmap.squeeze(), space).squeeze()
            #
            ## Load in results table.
            resultsf = op.join(label_dir, '%s' + ('_mni2.%s.%s.%s.%s.%s.%s.%s.%s.%s.csv' % 
                                                  (version, task, model_name,
                                                   analysis, epochs_type,
                                                   sm, fd, hemi, condition)))
            if space == 'mni305':
                results = read_csv(resultsf % 'volume')
                fscores = [image[i,j,k] for i,j,k in results[['cX','cY','cZ']].values]
                results['Fpre'] = fscores
                results.to_csv(resultsf % 'fstat_volume', index=False)
            else:
                results = read_csv(resultsf % 'surface')
                if not 'Fpre' in results.columns: results['Fpre'] = np.nan
                vertices = results.loc[[True if label.endswith(space) else False for label in results.Label],'V'].values
                for v in vertices: results.loc[results.V==v,'Fpre'] = image[v]
                results.to_csv(resultsf % 'fstat_surface', index=False)
    
print('Done.')

/autofs/space/karima_001/users/alex/software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide


Done.
